# Bruhat order and He order

Notes related to a project of Maria Fox, Deewang Bhamidipati, et al.

## Setup

In [1]:
# Load a few packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Setup global variables
q = 12 # We work with permutations of the numbers 1...q.
X = list(range(1,q+1))
print(X)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


## Working with permutations

I'm not aiming for great efficiency here, so I'm using Python functions to encode permutations.

In [3]:
def simp(i): # Creates the transposition switching i and i+1
    if (1 <= i) and (i <= q-1): # Avoid problems.
        def s(x):
            if x == i:
                return i+1
            elif x == i+1:
                return i
            else:
                return x
        return s
    else:
        raise ValueError('Adjacent transpositions must have i between 1 and q-1.')
        return None

In [11]:
def r_perm():  # Creates a random permutation.
    rc = np.random.choice(X,q,replace=False)
    def f(x):
        return rc[x-1]
    return f

In [18]:
def print_perm(f):
    df = pd.DataFrame(index=['After'], columns=X)
    for x in X:
        df[x] = [str(f(x))]
    display(df)
    return None

In [19]:
print_perm(r_perm())

,1,2,3,4,5,6,7,8,9,10,11,12
After,3,9,6,2,12,10,7,5,1,4,8,11


In [20]:
print_perm(simp(1)) # Should print the transposition switching 1,2.

,1,2,3,4,5,6,7,8,9,10,11,12
After,2,1,3,4,5,6,7,8,9,10,11,12


In [21]:
def compose(f,g): # Composes permutations f and g.
    def h(x):
        return f(g(x))
    return h

In [190]:
def twistconj(f, i): # Twisted conjugation of f by (i i+1)
    if i == 0: # Don't twist at all.  Convenient.
        return f 
    elif i == 1:
        wi = 1
    elif (i >= 3) and (i <= q-1):
        wi = q+2-i # Note (3 4) goes to (q-1 q)
    else:
        raise ValueError('Illegal i in twisted conjugation')
    g = compose(f, simp(wi))
    return compose( simp(i), g) # Returns simp(i) o f o simp(wi)

In [26]:
def inv(f): # Invert a permutation
    def h(x):
        dictinv = {f(y) : y for y in X}
        return dictinv[x]
    return h

Let's test our creation of random permutations and inverses.

In [28]:
r = r_perm() # Create a random permutation
print_perm(r) 
print_perm(inv(r)) 
print_perm(compose(r,inv(r))) # Hope it's the identity!

,1,2,3,4,5,6,7,8,9,10,11,12
After,11,2,3,7,9,4,5,6,1,10,12,8


,1,2,3,4,5,6,7,8,9,10,11,12
After,9,2,3,6,7,8,4,12,5,10,1,11


,1,2,3,4,5,6,7,8,9,10,11,12
After,1,2,3,4,5,6,7,8,9,10,11,12


Now we create the coset representatives $\gamma_{u,v}$.  

In [29]:
def g(u,v): # These are the gamma_{u,v} matrices.
    def f(x):
        if x == u:
            return 1
        elif x == v:
            return 2
        elif x <= u-1:
            return x+2
        elif x <= v-1:
            return x+1
        else:
            return x
    return f

In [30]:
def gi(u,v): # Sometimes nicer to work with their inverses.
    return inv(g(u,v))

In [33]:
print_perm(gi(3,7)) # 1 goes to 3, 3 goes to 7.

,1,2,3,4,5,6,7,8,9,10,11,12
After,3,7,1,2,4,5,6,8,9,10,11,12


# The $\sigma[i,j]$ matrices and Bruhat order

In [34]:
def make_ij(f): # A pretty inefficient approach, but who cares for now.
    M = pd.DataFrame(index=X, columns=X)
    for i in X:
        for j in X:
            M.loc[i,j] = len([x for x in X if ((x <= i) and (f(x) >= j))])
    return M

In [38]:
def Texdf(df): # For copypasta to LaTeX
    cstring = 'c||'+'c'*len(df.columns)
    print('$$\\begin{small}  \\begin{tabular}{'+cstring+'}')
    fline = '& ' + ' & '.join(list(df.columns.astype(str))) + ' \\\\ \\hline \\hline'
    print(fline)
    for i in df.index:
        row = df.loc[i]
        fline = str(i) + ' & ' + ' & '.join(list(row.astype(str))) + ' \\\\'
        print(fline)
    print('\\end{tabular} \\end{small}$$')

In [39]:
Texdf(make_ij(gi(2,10)))

$$\begin{small}  \begin{tabular}{c||cccccccccccc}
& 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 \\ \hline \hline
1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
2 & 2 & 2 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 0 & 0 \\
3 & 3 & 2 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 0 & 0 \\
4 & 4 & 3 & 2 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 0 & 0 \\
5 & 5 & 4 & 3 & 2 & 1 & 1 & 1 & 1 & 1 & 1 & 0 & 0 \\
6 & 6 & 5 & 4 & 3 & 2 & 1 & 1 & 1 & 1 & 1 & 0 & 0 \\
7 & 7 & 6 & 5 & 4 & 3 & 2 & 1 & 1 & 1 & 1 & 0 & 0 \\
8 & 8 & 7 & 6 & 5 & 4 & 3 & 2 & 1 & 1 & 1 & 0 & 0 \\
9 & 9 & 8 & 7 & 6 & 5 & 4 & 3 & 2 & 1 & 1 & 0 & 0 \\
10 & 10 & 9 & 8 & 7 & 6 & 5 & 4 & 3 & 2 & 1 & 0 & 0 \\
11 & 11 & 10 & 9 & 8 & 7 & 6 & 5 & 4 & 3 & 2 & 1 & 0 \\
12 & 12 & 11 & 10 & 9 & 8 & 7 & 6 & 5 & 4 & 3 & 2 & 1 \\
\end{tabular} \end{small}$$


In [41]:
make_ij(gi(4,6))

,1,2,3,4,5,6,7,8,9,10,11,12
1,1,1,1,1,0,0,0,0,0,0,0,0
2,2,2,2,2,1,1,0,0,0,0,0,0
3,3,2,2,2,1,1,0,0,0,0,0,0
4,4,3,2,2,1,1,0,0,0,0,0,0
5,5,4,3,2,1,1,0,0,0,0,0,0
6,6,5,4,3,2,1,0,0,0,0,0,0
7,7,6,5,4,3,2,1,0,0,0,0,0
8,8,7,6,5,4,3,2,1,0,0,0,0
9,9,8,7,6,5,4,3,2,1,0,0,0
10,10,9,8,7,6,5,4,3,2,1,0,0


In [42]:
def Bruhat(f,g): # True if f <= g in Bruhat order.
    M = make_ij(f)
    N = make_ij(g)
    diff = N - M
    return (diff.values.flatten().min() >= 0)


It's claimed that
$$\gamma_{u,v} \leq \gamma_{r,s} \Leftrightarrow (u \leq r) \text{and} (v \leq s).$$
Here we check this experimentally for a bunch of u,v,r,s quadruples.

In [46]:
for u in range(1,5):
    for v in range(u+1,12,3):
        for r in [1,2,4,8]:
            for s in [r+1, r+3]:
                B_bool = Bruhat(g(u,v),g(r,s))
                C_bool = ((u <= r) & (v <= s))
                print(u,v,r,s, B_bool, '=', C_bool, '?', B_bool == C_bool) # Hopefully all True on the right.

1 2 1 2 True = True ? True
1 2 1 4 True = True ? True
1 2 2 3 True = True ? True
1 2 2 5 True = True ? True
1 2 4 5 True = True ? True
1 2 4 7 True = True ? True
1 2 8 9 True = True ? True
1 2 8 11 True = True ? True
1 5 1 2 False = False ? True
1 5 1 4 False = False ? True
1 5 2 3 False = False ? True
1 5 2 5 True = True ? True
1 5 4 5 True = True ? True
1 5 4 7 True = True ? True
1 5 8 9 True = True ? True
1 5 8 11 True = True ? True
1 8 1 2 False = False ? True
1 8 1 4 False = False ? True
1 8 2 3 False = False ? True
1 8 2 5 False = False ? True
1 8 4 5 False = False ? True
1 8 4 7 False = False ? True
1 8 8 9 True = True ? True
1 8 8 11 True = True ? True
1 11 1 2 False = False ? True
1 11 1 4 False = False ? True
1 11 2 3 False = False ? True
1 11 2 5 False = False ? True
1 11 4 5 False = False ? True
1 11 4 7 False = False ? True
1 11 8 9 False = False ? True
1 11 8 11 True = True ? True
2 3 1 2 False = False ? True
2 3 1 4 False = False ? True
2 3 2 3 True = True ? True
2 3 2 5

## The effect of twisted conjugation on [i,j] matrices.

(These matrices must have a better name in the literature.)

The lines below are meant to test the effect of twisted conjugation on the [i,j] matrix, for a random starting matrix.  We perform twisted conjugation by a transposition $(\ell \ \ell+1)$ for $\ell = 1$ or $3 \leq \ell \leq q$.

In [57]:
rp = r_perm()
M_old = make_ij(rp)
M_new = make_ij(twistconj(rp, 6)) # Replace 5 by anything between 3 and q to see what happens.
display(M_old,  M_new - M_old) # Top matrix is original.  Bottom if difference between new and original.

,1,2,3,4,5,6,7,8,9,10,11,12
1,1,0,0,0,0,0,0,0,0,0,0,0
2,2,1,1,1,1,1,1,1,0,0,0,0
3,3,2,1,1,1,1,1,1,0,0,0,0
4,4,3,2,2,2,2,1,1,0,0,0,0
5,5,4,3,3,3,3,2,2,1,0,0,0
6,6,5,4,4,4,4,3,3,2,1,1,0
7,7,6,5,5,5,4,3,3,2,1,1,0
8,8,7,6,6,6,5,4,3,2,1,1,0
9,9,8,7,7,7,6,5,4,3,2,1,0
10,10,9,8,8,7,6,5,4,3,2,1,0


,1,2,3,4,5,6,7,8,9,10,11,12
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,1,1,1,1,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,0,0


It's clear that twisted conjugation by $\sigma_\ell$ has the effect of changing the $\ell+1$ column and the $\ell'$ row, where $\ell' = q+2-\ell$, when $3 \leq \ell \leq q-1$.  The changes seem to involve adding/subtracting a (typically consecutive) block of ones.  

In [60]:
rp = r_perm()
M_old = make_ij(rp)
M_new = make_ij(twistconj(rp, 1)) # Replace 5 by anything between 3 and q to see what happens.
display(M_old,  M_new - M_old) # Top matrix is original.  Bottom if difference between new and original.

,1,2,3,4,5,6,7,8,9,10,11,12
1,1,1,1,1,0,0,0,0,0,0,0,0
2,2,2,2,2,1,1,1,1,1,0,0,0
3,3,3,2,2,1,1,1,1,1,0,0,0
4,4,4,3,3,2,2,2,2,2,1,1,1
5,5,5,4,4,3,3,3,3,3,2,2,1
6,6,6,5,5,4,3,3,3,3,2,2,1
7,7,7,6,6,5,4,3,3,3,2,2,1
8,8,8,7,6,5,4,3,3,3,2,2,1
9,9,8,7,6,5,4,3,3,3,2,2,1
10,10,9,8,7,6,5,4,4,3,2,2,1


,1,2,3,4,5,6,7,8,9,10,11,12
1,0,0,0,0,1,1,1,1,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,-1,0,0,0,0,0,0,0,0,0,0
4,0,-1,0,0,0,0,0,0,0,0,0,0
5,0,-1,0,0,0,0,0,0,0,0,0,0
6,0,-1,0,0,0,0,0,0,0,0,0,0
7,0,-1,0,0,0,0,0,0,0,0,0,0
8,0,-1,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,0,0


Something similar occurs when performing twisted conjugation by $s_1$, i.e., $\sigma \mapsto s_1 \sigma s_1$.  This affects the first row and the second column.

Let's see the effect of twisted conjugation on $\gamma_{u,v}^{-1}$.

In [65]:
my_p = gi(4,9) # Generic... somewhat spaced out.
M_old = make_ij(my_p)
M_old

,1,2,3,4,5,6,7,8,9,10,11,12
1,1,1,1,1,0,0,0,0,0,0,0,0
2,2,2,2,2,1,1,1,1,1,0,0,0
3,3,2,2,2,1,1,1,1,1,0,0,0
4,4,3,2,2,1,1,1,1,1,0,0,0
5,5,4,3,2,1,1,1,1,1,0,0,0
6,6,5,4,3,2,1,1,1,1,0,0,0
7,7,6,5,4,3,2,1,1,1,0,0,0
8,8,7,6,5,4,3,2,1,1,0,0,0
9,9,8,7,6,5,4,3,2,1,0,0,0
10,10,9,8,7,6,5,4,3,2,1,0,0


Here I do a little brute-force searching to look for non-obvious He inequalities.

In [233]:
J = [1,3,4,5,6,7,8,9,10,11]
JJ = [0] + J # Include identity element in twists.  Twisting by 0 = doing nothing here.
my_p = gi(7,8)
comp_p = gi(6,9)
SS = [(a,b,c) for a in JJ for b in JJ for c in JJ] # Search length <= 3 words in J.
for a,b,c in SS:
    tj = twistconj(my_p, a)
    tj = twistconj(tj, b)
    tj = twistconj(tj, c)
    if Bruhat(tj, comp_p):
        print(a,b,c,'Yes!')
        break


I find some nice ones:  $\gamma_{r,s} \leq \gamma_{u,v}$ when...
$$(r,s) = (4,5) \text{ and } (u,v) = (3,12).$$
$$(r,s) = (5,6) \text{ and } (u,v) = (4,11).$$
$$(r,s) = (6,7) \text{ and } (u,v) = (5,10).$$


This does follow a nice little pattern.  All of these are possible with twisted-conjugation by a single element of $J$.  Perhaps all He inequalities arise from such "easy" ones together with the "obvious" ones arising from Bruhat order (without any twisted conjugation needed).  I don't know!  That's all I find looking up to length-three words.